# Working with trajectories

In the following, we demonstrate how to use coherent structure detection methods
that work directly on trajectory data sets. These include the graph
Laplace-based and the transfer operator-based methods for approximating the
dynamic Laplacian.

## Graph Laplace-based methods

In the following, we demonstrate how to apply several graph Laplace-based coherent
structure detection methods. For references and technical details, we refer to
the corresponding Graph Laplacian/diffusion maps-based LCS methods page.

As an example, this is how we can add more processes to run code in parallel.

In [ ]:
using Distributed
(nprocs() == 1) && addprocs()

We first load our package and some dependencies.

In [ ]:
@everywhere using CoherentStructures
using StaticArrays, Distances, Plots

Next, we define the usual flow parameters. For visualization convenience,
we use a regular grid at initial time.

In [ ]:
tspan = range(10*24*3600.0, stop=30*24*3600.0, length=41)
m = 120; n = 41; N = m*n
x = range(0.0, stop=20.0, length=m)
y = range(-3.0, stop=3.0, length=n)
f = u -> flow(bickleyJet, u, tspan, tolerance=1e-4)
particles = vec(SVector{2}.(x, y'))
trajectories = pmap(f, particles; batch_size=m)

The flow is defined on a cylinder with the following periods in x and y. The
variable `metric` defines the (spatial) distance metric.

In [ ]:
periods = [6.371π, Inf]
metric = PEuclidean(periods)

We would like calculate 6 diffusion coordinates for each example.

In [ ]:
n_coords = 6

We now illustrate some of the different graph Laplace-based methods, and simply
visualize some eigenvectors, mostly without further postprocessing.

### Spectral-clustering approach/L_1 time averaging [Hadjighasem, Karrasch, Teramoto & Haller, 2016]

In [ ]:
ε = 3e-1
kernel = gaussian(ε)
P = sparse_diff_op(trajectories, Neighborhood(gaussiancutoff(ε/5)), kernel; metric=STmetric(metric, 1))
λ, Ψ = diffusion_coordinates(P, n_coords)

We plot the second and third eigenvectors.

In [ ]:
field = permutedims(reshape(Ψ[:, 2], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

field = permutedims(reshape(Ψ[:, 3], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

Another sparsification option is k-nearest neighbors. The following is a
demonstration for 400 nearest, non-mutual neighbors. For the mutual nearest neighbors
sparsification, choose `MutualKNN()`.

In [ ]:
P = sparse_diff_op(trajectories, KNN(400), kernel; metric=STmetric(metric, Inf))
λ, Ψ = diffusion_coordinates(P, n_coords)

field = permutedims(reshape(Ψ[:, 2], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

field = permutedims(reshape(Ψ[:, 3], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

### Use of SEBA to extract features

For feature extraction from operator eigenvectors, one may use the "SEBA"
method developed by [Froyland, Rock & Sakellariou, 2019].

In [ ]:
Ψ2 = SEBA(Ψ)

We plot two of the SEBA features extracted.

In [ ]:
field = permutedims(reshape(Ψ2[:, 1], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

field = permutedims(reshape(Ψ2[:, 2], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

### Space-time diffusion maps [Banisch & Koltai, 2017]

In [ ]:
import Statistics: mean
ε = 1e-3
kernel = gaussian(ε)
P = sparse_diff_op_family(trajectories, Neighborhood(gaussiancutoff(ε)), kernel, mean; metric=metric)
λ, Ψ = diffusion_coordinates(P, n_coords)

field = permutedims(reshape(Ψ[:, 2], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

field = permutedims(reshape(Ψ[:, 3], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

### Network-based approach [Padberg-Gehle & Schneide, 2017]

In [ ]:
ε = 0.2
P = sparse_diff_op_family(trajectories, Neighborhood(ε), Base.one, P -> row_normalize!(min.(sum(P), 1));
                            α=0, metric=metric)
λ, Ψ = diffusion_coordinates(P, n_coords)

field = permutedims(reshape(Ψ[:, 2], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

field = permutedims(reshape(Ψ[:, 3], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

### Time coupled diffusion coordinates [Marshall & Hirn, 2018]

In [ ]:
ε = 1e-3
kernel = gaussian(ε)
P = sparse_diff_op_family(trajectories, Neighborhood(gaussiancutoff(ε)), kernel; metric=metric)
λ, Ψ = diffusion_coordinates(P, n_coords)

field = permutedims(reshape(Ψ[:, 2], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

field = permutedims(reshape(Ψ[:, 3], m, n))
fig = Plots.heatmap(x, y, field, aspect_ratio=1, color=:viridis)
Plots.plot(fig)

## FEM adaptive TO method

We first generate some trajectories on a set of `n` random points for the
rotating double gyre flow.

In [ ]:
using CoherentStructures, StaticArrays, Tensors

n = 500
tspan = range(0, stop=1.0, length=20)
xs, ys = rand(n), rand(n)
particles = SVector{2}.(xs, ys)
trajectories = [flow(rot_double_gyre, particles[i], tspan) for i in 1:n]

Based on the initial particle positions we generate a triangulation.
If this call fails or does not return, the initial positions may not be unique.
In that case, simply generate a different set of random initial positions.

In [ ]:
ctx, _ = irregularDelaunayGrid(Vec{2}.(particles))

Next, we generate the stiffness and mass matrices and solve the generalized
eigenproblem.

In [ ]:
S = adaptiveTOCollocationStiffnessMatrix(ctx, (i, ts) -> trajectories[i], tspan; flow_map_mode=1)
M = assembleMassMatrix(ctx)

using Arpack
λ, V = eigs(S, M; which=:SM, nev=6)

We can plot the spectrum obtained.

In [ ]:
using Plots
fig = plot_real_spectrum(λ)
Plots.plot(fig)

We may extract coherent vortices with k-means clustering.

In [ ]:
using Clustering
function iterated_kmeans(iterations, args...)
    best = kmeans(args...)
    for i in 1:(iterations - 1)
        cur = kmeans(args...)
        if cur.totalcost < best.totalcost
            best = cur
        end
    end
    return best.assignments
end

partitions = 3
clusters = iterated_kmeans(20, permutedims(V[:, 2:partitions]), partitions)

A simple scatter plot visualization looks as follows.

In [ ]:
fig = scatter(xs, ys, zcolor=clusters[ctx.node_to_dof], markersize=8)
Plots.plot(fig)

Alternatively, we may also plot the cluster assignments on the whole irregular
grid.

In [ ]:
fig = plot_u(ctx, float(clusters), 400, 400;
    color=:viridis, colorbar=:none, title="$partitions-partition of rotating double gyre")
Plots.plot(fig)

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*